This is an advanced example of model evaluation with the addition of POS tag vectorization (based on an other column in the data set), using FeatureUnion in a pipeline and GridSearchCV with cross-validation splitting strategy (5 folds to make it faster)

There is also an example of extracting most informative features for the prediction

See full version in:
https://github.com/KaterynaD/TweetsAutorshipAttributionModelsEvaluation/blob/master/POS%2Btag%2Bvectorization.ipynb

I use Bernoulli Naive Bayes classifier only because on these data all classifiers provides more or less the same results.
See https://github.com/KaterynaD/TweetsAutorshipAttributionModelsEvaluation/blob/master/Autorship%2Battribution%2Bmodels%2Bevaluation.ipynb

From the achieved results point of view: char and stemmed word vectorizeres provides good results. The other combinations (with or without POS tag vectorizer) may or may not provide slightly better or worse results.
For most pairs of authors I get 90 - 96% accuracy. I experimented with 500 - 2000 rows rows data sets per author

In [ ]:
import pandas as pd
from pandas import Series,DataFrame
import numpy as np

For sparsity reasons I pre-process data before analysis:

 1. removing re-tweets
 2. removing short messages (less then 4 words)
 3. replacing @ with REF
 4. replacing any url with URL
 5. replacing any date with DATE
 6. replacing any time with TIME
 7. replace digits with NUM

In [ ]:
#data
#KimKardashianTweets data
df_kk=pd.read_csv('../input/KimKardashianTweets.csv')
len(df_kk)

In [ ]:
#HillaryClintonTweets data
df_hc=pd.read_csv('../input/HillaryClintonTweets.csv')
len(df_hc)

In [ ]:
author1='KimKardashian'
author2='HillaryClinton'

In [ ]:
import random
#2000 random sample rows for KK
rows = random.sample(list(df_kk.index), 2000)
df_kk = df_kk.ix[rows]
#2000 random sample rows for HC
rows = random.sample(list(df_hc.index), 2000)
df_hc = df_hc.ix[rows]
#join back together
df=df_kk.append(df_hc,ignore_index=True)
len(df)

In [ ]:
#data pre-processing
df.drop(df[df.retweet==True].index, inplace=True)
df['num_of_words'] = df["text"].str.split().apply(len)
df.drop(df[df.num_of_words<4].index, inplace=True)
df["text"].replace(r"http\S+", "URL", regex=True,inplace=True)
df["text"].replace(r"@\S+", "REF", regex=True ,inplace=True)
df["text"].replace(r"(\d{1,2})[/.-](\d{1,2})[/.-](\d{2,4})+", "DATE", regex=True,inplace=True)
df["text"].replace(r"(\d{1,2})[/:](\d{2})[/:](\d{2})?(am|pm)+", "TIME", regex=True,inplace=True)
df["text"].replace(r"(\d{1,2})[/:](\d{2})?(am|pm)+", "TIME", regex=True,inplace=True)
df["text"].replace(r"\d+", "NUM", regex=True,inplace=True)
len(df)

POS tag vectorizer

I am going to convert the text column in the data set to the string of POS tag and replace the usual POS tag 2-3 chars abbreviations to 1 char abbrevations (e.g. 'NNP' -> 'N', 'NNPS' -> 'O') in order to use CountVectorizer with 'char' analyzer to get the most informative POS tag combinations per author

In [ ]:
#POS tag 2-3 chars abbrivation mapping to 1 char abbrevations
#http://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html
pos_code_map={'CC':'A','CD':'B','DT':'C','EX':'D','FW':'E','IN':'F','JJ':'G','JJR':'H','JJS':'I','LS':'J','MD':'K','NN':'L','NNS':'M',
'NNP':'N','NNPS':'O','PDT':'P','POS':'Q','PRP':'R','PRP$':'S','RB':'T','RBR':'U','RBS':'V','RP':'W','SYM':'X','TO':'Y','UH':'Z',
'VB':'1','VBD':'2','VBG':'3','VBN':'4','VBP':'5','VBZ':'6','WDT':'7','WP':'8','WP$':'9','WRB':'@'}
#Python 2 code_pos_map={v: k for k, v in pos_code_map.iteritems()}
code_pos_map = {v: k for k, v in  pos_code_map.items()}

In [ ]:
#abbrivation converters
def convert(tag):
    try:
        code=pos_code_map[tag]
    except:
        code='?'
    return code
def inv_convert(code):
    try:
        tag=code_pos_map[code]
    except:
        tag='?'
    return tag

In [ ]:
#POS tag converting
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk import pos_tag, word_tokenize
def pos_tags(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text_processed=tokenizer.tokenize(text)
    return "".join(convert(tag) for (word, tag) in nltk.pos_tag(text_processed))
def text_pos_inv_convert(text):
    return "-".join(inv_convert(c.upper()) for c in text)

In [ ]:
#a new column for pos tags
df['text_pos']=df.apply(lambda x: pos_tags(x['text']), axis=1)

Here is how a sequence of pos tags looks like to be used in CountVectorizer()

In [ ]:
df.ix[:,['author','text','text_pos']].head()

Now let's look if there are unique combinations of POS tags per author

In [ ]:
df_features=pd.DataFrame()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
for a in df.author.unique():
    v = CountVectorizer(analyzer='char',ngram_range=(3, 3))
    ngrams = v.fit_transform(df[df['author'] == a]['text_pos'])
    df_t=pd.DataFrame(
    {'Feature': v.get_feature_names(),
     'Count': list(ngrams.sum(axis=0).flat),
     'Author': a
    })
    #
    df_features=df_features.append(df_t,ignore_index=True)

Let's convert the 1 char abbrivated pos tag sequence back to the common known 2-3 chars abbreviations

In [ ]:
df_features['Feature_POS']=df_features.apply(lambda x: text_pos_inv_convert(x['Feature']), axis=1)

There are indeed a lot of unique POS tags combinations per author

In [ ]:
df_features[~df_features.Feature.isin(df_features[df_features['Author'] != author2].Feature)].sort_values('Count', ascending=False).ix[:,['Author','Count','Feature','Feature_POS']].head()

In [ ]:
df_features[~df_features.Feature.isin(df_features[df_features['Author'] != author1].Feature)].sort_values('Count', ascending=False).ix[:,['Author','Count','Feature','Feature_POS']].head()

To avoid overfitting let's hold out a part of the available data as a test set twt_test (X), author_test (Y).

In [ ]:
from sklearn.model_selection import train_test_split
twt_train, twt_test, author_train, author_test = train_test_split(df.ix[:,['text','text_pos']], df['author'], test_size=0.4, random_state=42)

The function will be used as tokenizer in the evaluation. As I discovered using stop words does not improve the model so I removed item 2 (removing stop words)

In [ ]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
def text_process(text):
    """
    Takes in a string of text, then performs the following:
    1. Tokenizes and removes punctuation
    3. Stems
    4. Returns a list of the cleaned text
    """

    # tokenizing
    tokenizer = RegexpTokenizer(r'\w+')
    text_processed=tokenizer.tokenize(text)
    
    
    # steming
    porter_stemmer = PorterStemmer()
    
    text_processed = [porter_stemmer.stem(word) for word in text_processed]
    

    return text_processed

In [ ]:
ScoreSummaryByModelParams = list()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

ItemSelector and TextAndTextCodedExtractor classes is used in a pipeline to get a proper column (text or text_pos) from a data set to be used in a vectorizer

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
class ItemSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]

In [ ]:
class TextAndTextCodedExtractor(BaseEstimator, TransformerMixin):
    """Extract the text & text_pos from a tweet in a single pass.
    """
    def fit(self, x, y=None):
        return self

    def transform(self, tweets):
        features=tweets.ix[:,['text_pos','text']].to_records(index=False)

        return features

ModelParamsEvaluation function receives as parameters 2 parts of its pipeline: f_union, which is a pipeline itself with different combinations of vectorizers and a model

In [ ]:
def ModelParamsEvaluation (f_union,model,params,comment):
    pipeline = Pipeline([
    # Extract the text & text_coded
    ('textandtextcoded', TextAndTextCodedExtractor()),

    # Use FeatureUnion to combine the features from text and text_coded
    ('union', f_union, ),

    # Use a  classifier on the combined features
    ('clf', model),
    ])
    grid_search = GridSearchCV(estimator=pipeline, param_grid=params, verbose=1, cv=5)
    grid_search.fit(twt_train, author_train)
    #best score
    print("Best score: %0.3f" % grid_search.best_score_)
    print("Best parameters set:")
    best_parameters = grid_search.best_estimator_.get_params()
    for param_name in sorted(params.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))
        ScoreSummaryByModelParams.append([comment,grid_search.best_score_,"\t%s: %r" % (param_name, best_parameters[param_name])])    
 

First I examine the model with only 1 vectorizer We do not need FeatureUnion in this case but I use it just to keep the pattern of all experiments

In [ ]:
f1_union=FeatureUnion(
        transformer_list=[
              # Pipeline for pulling char features  from the text
            ('char', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='char')),
            ])),               

        ],
    )

'char' analyzer provides a perfect result by itself

In [ ]:
from sklearn.naive_bayes import BernoulliNB
p = {
    'union__char__tfidf__max_df': (0.5, 0.75, 1.0),
    'union__char__tfidf__ngram_range': ((2, 2), (3, 3)), 
    'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}

ModelParamsEvaluation(f1_union,BernoulliNB(),p,'Bernoulli Naive Bayes, char')

In [ ]:
f1_union=FeatureUnion(
        transformer_list=[
            # Pipeline for pulling word features from the text
            ('word', Pipeline([
            ('selector', ItemSelector(key='text')),
            ('tfidf',    TfidfVectorizer(analyzer='word')),
            ])),              

        ],
    )

'word' analyzer is worse for these 2 authors but for other pairs (AdamSavage - ScottKelly) it provides better results the the char analyzer. As you can see it is not recommended using stop words

In [ ]:
p = {
    'union__word__tfidf__max_df': (0.5, 0.75, 1.0),
    'union__word__tfidf__ngram_range': ((1, 1),(2, 2), (3, 3)), 
    'union__word__tfidf__stop_words': (None, 'english'),
    'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}

ModelParamsEvaluation(f1_union,BernoulliNB(),p,'Bernoulli Naive Bayes, word')

In [ ]:
f1_union=FeatureUnion(
        transformer_list=[
            # Pipeline for pulling word features from the text
            ('text', Pipeline([
            ('selector', ItemSelector(key='text')),
            ('tfidf',    TfidfVectorizer(analyzer='word',tokenizer= text_process)),
            ])),              

        ],
    )

'stemmed word' analyzer is better then the just 'word' analyzer but still worse then the 'char' for these 2 authors
But for other pairs (AdamSavage - ScottKelly) it provides better results the the char analyzer and worse then the just 'word'

In [ ]:
p = {
    'union__text__tfidf__max_df': (0.5, 0.75, 1.0),
    'union__text__tfidf__ngram_range': ((1, 1),(2, 2), (3, 3)), 
    'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}

ModelParamsEvaluation(f1_union,BernoulliNB(),p,'Bernoulli Naive Bayes, stemmed words, no stop words')

In [ ]:
f1_union=FeatureUnion(
        transformer_list=[
            # Pipeline for pulling pos tag features  from the text_pos
            ('text_pos', Pipeline([
            ('selector', ItemSelector(key='text_pos')),
            ('tfidf',    TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=None)),
            ])),                  

        ],
    )

POS tag vectorizer is not very selective. Let's see how it words in the combinations

In [ ]:
p = {
    'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}

ModelParamsEvaluation(f1_union,BernoulliNB(),p,'Bernoulli Naive Bayes, POS tags')

In [ ]:
df_ScoreSummaryByModelParams=DataFrame(ScoreSummaryByModelParams,columns=['Method','BestScore','BestParameter'])
df_ScoreSummaryByModelParams.sort_values(['BestScore'],ascending=False,inplace=True)
df_ScoreSummaryByModelParams

With small variations 'char and stemmed word' combination provides the best result for most of analyzed authors pairs

Using POS tag vectorizer does not improve the score dramatically. Its impact is more visible only for AdamSavage - ScottKelly pair

In [ ]:
f3_union=FeatureUnion(
        transformer_list=[
             # Pipeline for pulling word stemmed features from the text
            ('text', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='word',tokenizer= text_process,ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),
                    
            # Pipeline for pulling char features  from the text
            ('char', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=5000)),
            ])),
                    
            # Pipeline for pulling flexible pattern features  from the text_coded with POS tags
            ('text_pos', Pipeline([
                ('selector', ItemSelector(key='text_pos')),
                ('tfidf',    TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=None)),
            ])),                  

        ],

    )

In [ ]:
p = {'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}
ModelParamsEvaluation(f3_union,BernoulliNB(),p,'Bernoulli Naive Bayes, char + stemmed word + POS tags')

In [ ]:
df_ScoreSummaryByModelParams=DataFrame(ScoreSummaryByModelParams,columns=['Method','BestScore','BestParameter'])
df_ScoreSummaryByModelParams.sort_values(['BestScore'],ascending=False,inplace=True)
df_ScoreSummaryByModelParams

Now let's run prediction and review the results
PredictionEvaluation function combines the scores from several methods for comparizon, ModelRun function runs the prediction for different models and most_informative_feature_for_binary_classification is used to get most informative features from a model

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc,precision_score, accuracy_score, recall_score, f1_score
from scipy import interp

In [ ]:
ScoreSummaryByVector = list()

In [ ]:
def PredictionEvaluation(author_test_b,author_predictions_b,comment):
    Precision=precision_score(author_test_b,author_predictions_b)
    print ('Precision: %0.3f' % (Precision))
    Accuracy=accuracy_score(author_test_b,author_predictions_b)
    print ('Accuracy: %0.3f' % (Accuracy))
    Recall=recall_score(author_test_b,author_predictions_b)
    print ('Recall: %0.3f' % (Recall))
    F1=f1_score(author_test_b,author_predictions_b)
    print ('F1: %0.3f' % (F1))
    print ('Confussion matrix:')
    print (confusion_matrix(author_test_b,author_predictions_b))
    ROC_AUC=roc_auc_score(author_test_b,author_predictions_b)
    print ('ROC-AUC: %0.3f' % (ROC_AUC))
    ScoreSummaryByVector.append([Precision,Accuracy,Recall,F1,ROC_AUC,comment])

In [ ]:
def ModelRun (f_union,model):
    pipeline = Pipeline([
    # Extract the text & text_coded
    ('textandtextcoded', TextAndTextCodedExtractor()),

    # Use FeatureUnion to combine the features from text and text_coded
    ('union', f_union, ),

    # Use a  classifier on the combined features
    ('clf', model),
    ])
    pipeline.fit(twt_train, author_train)
    author_predicted = pipeline.predict(twt_test)
    
    feature_names=list()
    for p in (pipeline.get_params()['union'].transformer_list):
        fn=(p[0],pipeline.get_params()['union'].get_params()[p[0]].get_params()['tfidf'].get_feature_names())
        feature_names.append(fn)
    df_fn=pd.DataFrame()
    for fn in feature_names:
        df_fn= df_fn.append(pd.DataFrame(
        {'FeatureType': fn[0],
         'Feature': fn[1]
        }),
        ignore_index=True)    
    
    from sklearn.preprocessing import LabelBinarizer
    lb = LabelBinarizer()
    author_test_b = lb.fit_transform(author_test.values)
    author_predicted_b  = lb.fit_transform(author_predicted)
    return (df_fn,pipeline.get_params()['clf'],author_predicted,author_predicted_b, author_test_b)

In [ ]:
def most_informative_feature_for_binary_classification(feature_names, classifier):
    class_labels = classifier.classes_

    topnvalues_class0 = sorted(zip(classifier.coef_[0], feature_names['Feature'].values, feature_names['FeatureType'].values))
    topnvalues_class1 = sorted(zip(classifier.coef_[0], feature_names['Feature'].values, feature_names['FeatureType'].values), reverse=True)

    topn_df_class0=pd.DataFrame(topnvalues_class0, columns=['Coef','Feature','FeatureType'])
    topn_df_class0['Author']=class_labels[0]
    
    topn_df_class1=pd.DataFrame(topnvalues_class1, columns=['Coef','Feature','FeatureType'])
    topn_df_class1['Author']=class_labels[1]    
    
    topn_df=topn_df_class0.append(topn_df_class1)
    
        
    return topn_df

In [ ]:
f2_union=FeatureUnion(
        transformer_list=[
            # Pipeline for pulling char features  from the text
            ('char', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=5000)),
            ])),
            # Pipeline for pulling stememd word features from the text
            ('text', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',    TfidfVectorizer(analyzer='word',tokenizer= text_process,ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),        

        ],

    )

In [ ]:
(feature_names,clf,author_predicted,author_predicted_b, author_test_b)=ModelRun(f2_union,BernoulliNB(alpha=0.0001))

In [ ]:
PredictionEvaluation(author_predicted_b, author_test_b,'char+stemmed word')

In [ ]:
f3_union=FeatureUnion(
        transformer_list=[
             # Pipeline for pulling word stemmed features from the text
            ('text', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='word',tokenizer= text_process,ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),
                    
            # Pipeline for pulling char features  from the text
            ('char', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=5000)),
            ])),
                    
            # Pipeline for pulling flexible pattern features  from the text_coded with POS tags
            ('text_pos', Pipeline([
                ('selector', ItemSelector(key='text_pos')),
                ('tfidf',    TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=None)),
            ])),                  

        ],

    )

In [ ]:
(feature_names,clf,author_predicted,author_predicted_b, author_test_b)=ModelRun(f3_union,BernoulliNB(alpha=0.0001))

In [ ]:
PredictionEvaluation(author_predicted_b, author_test_b,'char+stemmed word+POS tag')

In [ ]:
f4_union=FeatureUnion(
        transformer_list=[

            # Pipeline for pulling word features from the text
            ('word', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',    TfidfVectorizer(analyzer='word',stop_words=None,ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),
                    
             # Pipeline for pulling word features after word_processing from the text
            ('text', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='word',tokenizer= text_process,ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),
                    
            # Pipeline for pulling char features  from the text
            ('char', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=5000)),
            ])),
                    
            # Pipeline for pulling flexible pattern features  from the text_coded
            ('text_pos', Pipeline([
                ('selector', ItemSelector(key='text_pos')),
                ('tfidf',    TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=None)),
            ])),                  

        ],

    )

In [ ]:
(feature_names,clf,author_predicted,author_predicted_b, author_test_b)=ModelRun(f4_union,BernoulliNB(alpha=0.0001))

In [ ]:
PredictionEvaluation(author_predicted_b, author_test_b,'char+word+stemmed word+POS tag')

Here is the summary per model.

In [ ]:
df_ScoreSummaryByVector=DataFrame(ScoreSummaryByVector,columns=['Precision','Accuracy','Recall','F1','ROC-AUC','Vector'])
df_ScoreSummaryByVector.sort_values(['F1'],ascending=False,inplace=True)
df_ScoreSummaryByVector

Now let's review teh most informative features for the last prediction

In [ ]:
TopFeatures_df=most_informative_feature_for_binary_classification(feature_names, clf)

In [ ]:
df1=TopFeatures_df.loc[((TopFeatures_df['Author']==author2) & (TopFeatures_df['FeatureType']=='char')),['Author','Coef','Feature']].head(10)
df1.rename(columns={'Coef':'CoefChar','Feature':'Char'}, inplace=True)
df1.reset_index(inplace=True)
df2=TopFeatures_df.loc[((TopFeatures_df['Author']==author2) & (TopFeatures_df['FeatureType']=='word')),['Coef','Feature']].head(10)
df2.rename(columns={'Coef':'CoefWord','Feature':'Word'}, inplace=True)
df2.reset_index(inplace=True)
df3=TopFeatures_df.loc[((TopFeatures_df['Author']==author2) & (TopFeatures_df['FeatureType']=='text')),['Coef','Feature']].head(10)
df3.rename(columns={'Coef':'CoefText','Feature':'Text'}, inplace=True)
df3.reset_index(inplace=True)
df4=TopFeatures_df.loc[((TopFeatures_df['Author']==author2) & (TopFeatures_df['FeatureType']=='text_pos')),['Coef','Feature']].head(10)
df4.rename(columns={'Coef':'CoefTextPOS','Feature':'TextPOS'}, inplace=True)
df4['TextPOS']=df4.apply(lambda x: text_pos_inv_convert(x['TextPOS']), axis=1)
df4.reset_index(inplace=True)
df_kk_top_features = pd.concat([df1,df2,df3,df4],axis=1)
df_kk_top_features.drop('index', axis=1, inplace=True)
df_kk_top_features

In [ ]:
df1=TopFeatures_df.loc[((TopFeatures_df['Author']==author1) & (TopFeatures_df['FeatureType']=='char')),['Author','Coef','Feature']].head(10)
df1.rename(columns={'Coef':'CoefChar','Feature':'Char'}, inplace=True)
df1.reset_index(inplace=True)
df2=TopFeatures_df.loc[((TopFeatures_df['Author']==author1) & (TopFeatures_df['FeatureType']=='word')),['Coef','Feature']].head(10)
df2.rename(columns={'Coef':'CoefWord','Feature':'Word'}, inplace=True)
df2.reset_index(inplace=True)
df3=TopFeatures_df.loc[((TopFeatures_df['Author']==author1) & (TopFeatures_df['FeatureType']=='text')),['Coef','Feature']].head(10)
df3.rename(columns={'Coef':'CoefText','Feature':'Text'}, inplace=True)
df3.reset_index(inplace=True)
df4=TopFeatures_df.loc[((TopFeatures_df['Author']==author1) & (TopFeatures_df['FeatureType']=='text_pos')),['Coef','Feature']].head(10)
df4.rename(columns={'Coef':'CoefTextPOS','Feature':'TextPOS'}, inplace=True)
df4['TextPOS']=df4.apply(lambda x: text_pos_inv_convert(x['TextPOS']), axis=1)
df4.reset_index(inplace=True)
df_kk_top_features = pd.concat([df1,df2,df3,df4],axis=1)
df_kk_top_features.drop('index', axis=1, inplace=True)
df_kk_top_features

And let's take a look what was predicted wrongly

In [ ]:
author_predicted=pd.DataFrame(author_predicted,columns=['predicted'])
df_wrong_result = pd.concat([twt_test.reset_index(),author_test.reset_index(),author_predicted], axis=1)
df_wrong_result.drop('index', axis=1, inplace=True)
df_wrong_result.drop('text_pos', axis=1, inplace=True)
df_wrong_result=df_wrong_result[df_wrong_result['author']!=df_wrong_result['predicted']]
df_wrong_result.head(10)